In [1]:
import sys
sys.path.append('/fsx/matzeni/duck')

In [2]:
import torch
import h5py
import json
from pathlib import Path
import pickle
from tqdm import tqdm
import logging
from transformers import AutoTokenizer
import numpy as np
import faiss

from mblink.utils.utils import EntityCatalogue
from duck.datamodule import RelationCatalogue
from duck.common.utils import load_json
from duck.preprocessing.duck_index import DuckIndex
from duck.common.utils import load_json

logger = logging.getLogger()

In [3]:
with open('/fsx/matzeni/data/duck/duck_neighbors.json', 'r') as f:
    duck_neighbors = json.load(f)

In [4]:
def print_neighbors(entity, k=None):
    neighbors = duck_neighbors[entity]
    if k is not None:
        neighbors = neighbors[:k]
    neighbors_str = "; ".join(neighbors)
    print(f"{entity}: {neighbors_str}")

In [10]:
print_neighbors("Italy")

Italy: Portugal; Spain; Norway; Greece; Poland; Denmark; Belgium; Hungary; Finland; Republic of Ireland; Argentina; Brazil; Croatia; Mexico; Lithuania; Slovenia; Estonia; Switzerland; Sweden; Indonesia; Bulgaria; Serbia; Chile; Austria; Iceland; Iran; Turkey; Egypt; New Zealand; Syria; Slovakia; Cuba; Georgia (country); Latvia; Lebanon; Colombia; Ukraine; South Africa; Romania; Malta; Morocco; Montenegro; Czech Republic; Vietnam; Bolivia; Mongolia; Uruguay; North Macedonia; Kyrgyzstan; Cyprus


In [11]:
print_neighbors("Donald Trump")

Donald Trump: Joe Biden; Mrs. Bill Clinton; Barack Obama's; George W. Bush; Al Gore; Kamala Harris; Elizabeth Warren; Bill Clinton; Michael Bloomberg; Benjamin Netanyahu; Boris Johnson; Elon Musk; Vladimir Putin; Angela Merkel; Bill Gates; Xi Jinping; Bernie Sanders; Mitt Romney; Gavin Newsom; Arnold Schwarzenegger; John Kerry; George H. W. Bush; Nancy Pelosi; Justin Trudeau; John McCain; Jimmy Carter; Nicolas Sarkozy; Michelle Obama; Emmanuel Macron; Narendra Modi; Julian Assange; Recep Tayyip Erdoğan; François Hollande; Jeb Bush; Tom Hanks; Mike Pence; Melania Trump; Shinzō Abe; Ivanka Trump; Ben Affleck; Jane Goodall; Jeff Bezos; Alexandria Ocasio-Cortez; Shimon Peres; Rudy Giuliani; Tony Blair; Michael Moore; Mark Zuckerberg; Leonardo DiCaprio; Madeleine Albright


In [12]:
print_neighbors("Cristiano Ronaldo")

Cristiano Ronaldo: Lionel Messi; Luis Suárez; Gerard Piqué; Neymar; Manuel Neuer; Paul Pogba; Ronaldinho; Luka Modrić; Antoine Griezmann; Gareth Bale; Edinson Cavani; Mohamed Salah; Gianluigi Buffon; Leroy Sané; Didier Drogba; Thibaut Courtois; Karim Benzema; David Luiz; Xabi Alonso; Franck Ribéry; Andrés Iniesta; Romelu Lukaku; Harry Kane; Jordi Alba; Aymeric Laporte; Cesc Fàbregas; Zlatan Ibrahimović; Carles Puyol; Sergio Ramos; Jamie Carragher; Mario Götze; Raheem Sterling; Juan Mata; Sergio Agüero; Olivier Giroud; Georginio Wijnaldum; Kyle Walker; Ander Herrera; Hugo Lloris; Mario Balotelli; Samuel Eto'o; Gabriel Jesus; Kevin De Bruyne; Gianluigi Donnarumma; David Alaba; N'Golo Kanté; Ricardo Quaresma; Idrissa Gueye; Steve Mandanda; Kaká


In [13]:
print_neighbors("Justin Bieber")

Justin Bieber: Harry Styles; Chris Brown; Ed Sheeran; Eminem; Camila Cabello; Nick Jonas; Jordin Sparks; Richard Marx; Niall Horan; Shawn Mendes; Ne-Yo; Carrie Underwood; Usher (musician); Lil Wayne; Frank Ocean; Bruno Mars; Tim McGraw; Wiz Khalifa; Lorde; Zayn Malik; Iggy Azalea; Stevie Nicks; Ludacris; Calvin Harris; Nas; Liam Payne; Jason Derulo; Nicki Minaj; Sia (musician); Slash (musician); Enrique Iglesias; Halsey (singer); Sara Bareilles; Akon; Todd Rundgren; Ryan Adams; Michael Bublé; Jessie J; Joe Satriani; Kid Cudi; Keith Urban; John Mayer; Justin Timberlake; "Weird Al" Yankovic; Donald Glover; Skrillex; Kendrick Lamar; Sam Smith (singer); Busta Rhymes; Trey Songz


In [14]:
print_neighbors("London")

London: Istanbul; Madrid; Istanbul Province; Stockholm; Cairo; Buenos Aires; Manchester; Amsterdam; Milan; Ankara; Bern; Dublin; Mumbai; Sarajevo; Athens; Lisbon; Warsaw; Lyon; Belgrade; İzmir; Barcelona; Mexico City; Copenhagen; Birmingham; Marseille; Kolkata; Oslo; Ottawa; Helsinki; Beijing; Liverpool; Melbourne; Hamburg; Bangkok; Zürich; Caracas; Zagreb; Sydney; Vienna; Cape Town; Tehran; Munich; Toronto; Los Angeles; Florence; Budapest; Baghdad; Cardiff; Oxford; Geneva


In [15]:
print_neighbors("Rome")

Rome: Milan; Florence; Naples; Venice; Turin; Palermo; Rio de Janeiro; Genoa; Bologna; Lisbon; Amsterdam; Boston; Buenos Aires; Warsaw; Madrid; Istanbul; Barcelona; Athens; Bangkok; Cologne; Berlin; Istanbul Province; Los Angeles; Stockholm; Dublin; Lyon; Chicago; Cairo; Gdańsk; Belgrade; Padua; Hamburg; Dijon; Mexico City; London; Mumbai; Sydney; Bern; Sarajevo; Pisa; Jakarta; Helsinki; Havana; Brescia; Vilnius; Seoul; Rimini; Zürich; Geneva; Trieste


In [16]:
print_neighbors("Lion")

Lion: Tiger; Leopard; Cheetah; Jaguar; Red panda; Giant panda; Cougar; Okapi; Hippopotamus; Fennec fox; Chimpanzee; Capybara; Spotted hyena; Northern giraffe; Serval; Aye-aye; Llama; Snow leopard; Binturong; Aardvark; Meerkat; Aardwolf; Japanese macaque; Vicuña; Tayra; Ring-tailed lemur; Giant otter; Coyote; Yellow-throated marten; Ocelot; Bornean orangutan; Golden lion tamarin; Asian elephant; Caracal; South American tapir; Rhinoceros; Honey badger; Clouded leopard; Bonobo; Giant anteater; Pan (genus); African wild dog; Southern pig-tailed macaque; Jaguarundi; Common squirrel monkey; Fossa (animal); Crab-eating macaque; African bush elephant; Springbok; Kiang


In [17]:
print_neighbors("Jaguar")

Jaguar: Cougar; Ocelot; Giant anteater; Giant armadillo; Chimpanzee; Bonobo; Indian rhinoceros; Giant otter; Black rhinoceros; Pronghorn; White-lipped peccary; Maned wolf; Aye-aye; Aardwolf; Cheetah; Fennec fox; Linnaeus's two-toed sloth; South American tapir; Kinkajou; Margay; Black-footed cat; Aardvark; Mandrill; Bush dog; Tayra; Bornean orangutan; Serval; Collared peccary; Golden lion tamarin; Red panda; Arabian oryx; Virginia opossum; Okapi; Oncilla; Nilgai; Four-horned antelope; Leopard; Bobcat; Mountain tapir; Northern giraffe; Common squirrel monkey; Springbok; Common marmoset; Grévy's zebra; Spectacled bear; Meerkat; Caracal; Clouded leopard; Yellow-throated marten; Star-nosed mole


In [18]:
print_neighbors("Jaguar Cars")

Jaguar Cars: Land Rover; Steyr-Daimler-Puch; MG Cars; Gulf Oil; British Motor Corporation; Safeway (UK); Rover Group; Rover Company; MIPS Technologies; F. W. Woolworth Company; MG Rover Group; Rolls-Royce Motors; SS Cars; Austro-Daimler; Neue Automobil Gesellschaft; Bristol Siddeley; Hughes Helicopters; Westland Helicopters; Airbus Military; Avro Canada; Karl Kässbohrer Fahrzeugwerke; Fiat Aviazione; Rootes Group; TSR (company); Deutsche Werke; Force Protection Europe; Allied Corporation; Deutsche Werft; Saviem; Maxtor; Orange Armenia; Handspring (company); Tandberg; Logica; Galerías Preciados; Nash Motors; Brixia-Zust; Aeritalia; DASA; Uterqüe; Upjohn; North British Locomotive Company; Ling-Temco-Vought; Junkers; Humber Limited; Hamburger Flugzeugbau; DePatie–Freleng Enterprises; Martin Marietta; Alliance & Leicester; TRW Inc.


In [19]:
print_neighbors("Maserati")

Maserati: Lancia; VinFast; McLaren Automotive; Massimo Dutti; Paper Mate; Infiniti; Peroni Brewery; Lotus Cars; Colruyt (supermarket); Overkill Software; Panavia Aircraft GmbH; Dodge; Abarth; Church's Chicken; Freightliner Trucks; B&Q; Popeyes; Roger Dubuis; Alfred Dunhill Ltd.; Baskin-Robbins; DC Shoes; Gant (retailer); Neoplan; Ibanez; Sofitel; Harman Kardon; Stone Island; Au (mobile phone company); Helibras; Mouser Electronics; Mercedes-AMG; Alfa Romeo; Indian Motocycle Manufacturing Company; Yakovlev; Cadillac; Thorntons; Karma Automotive; DenizBank; Eastpak; Primark; Mercedes-Benz USA; Sandora; Crossword Bookstores; Ymobile Corporation; Franprix; Buick; Ikatel; Jackson Guitars; Super 8 Motels; Kobo Inc.


In [20]:
print_neighbors("Alan Turing")

Alan Turing: Bernhard Riemann; Kurt Gödel; John von Neumann; Herbert A. Simon; John Forbes Nash Jr.; Claude Shannon; Nikolai Lobachevsky; Benoit Mandelbrot; Willard Van Orman Quine; Niels Henrik Abel; Frank P. Ramsey; Hermann Minkowski; John Horton Conway; L. E. J. Brouwer; Gottlob Frege; Jan Łukasiewicz; Alfred Tarski; Jean Dieudonné; Hilary Putnam; Edmund Landau; Srinivasa Ramanujan; Emmy Noether; George Dantzig; B. F. Skinner; Évariste Galois; Alonzo Church; Øystein Ore; Dennis Ritchie; C. I. Lewis; André Weil; Georg Cantor; Harold Davenport; John Maynard Smith; John Edensor Littlewood; Carl Gustav Hempel; Giuseppe Peano; John Tukey; Rosalind Franklin; Kazimierz Twardowski; Lars Ahlfors; Erwin Schrödinger; Andrey Nikolayevich Tikhonov; Julian Schwinger; Saul Kripke; Eugen Dühring; Aleksandr Lyapunov; John Pople; Paul Cohen; Karl Pearson; Stefan Banach


In [21]:
print_neighbors("Gianluigi Buffon")

Gianluigi Buffon: Lorenzo Insigne; Leonardo Bonucci; Dries Mertens; Giorgio Chiellini; Leonardo Spinazzola; Bastian Schweinsteiger; Alessandro Florenzi; Salvatore Sirigu; Ciro Immobile; Mauro Icardi; Federico Chiesa; Riccardo Montolivo; Vincenzo Grifo; Kevin-Prince Boateng; Thomas Meunier; Daniele De Rossi; Marco Verratti; Gianluigi Donnarumma; Mesut Özil; Andrea Belotti; Mario Götze; Thorgan Hazard; Carles Puyol; Fabio Quagliarella; Toni Kroos; Milan Škriniar; Jérémy Ménez; Sead Kolašinac; Filippo Inzaghi; Florian Thauvin; Marco Reus; Pietro Pellegri; Phil Foden; Mats Hummels; Benjamin Henrichs; Alberto Aquilani; Alex Sandro; Franck Kessié; Manuel Locatelli; Lucas Digne; Edinson Cavani; Séamus Coleman; Mattia De Sciglio; André Onana; Federico Valverde; Franck Ribéry; Kevin Trapp; Milot Rashica; Marco Materazzi; Yann Sommer


In [23]:
print_neighbors("Ada Lovelace")

Ada Lovelace: Lady Byron; Rosalind Franklin; Elizabeth Fry; Catherine Dickens; Rosina Bulwer Lytton; Lady Emmeline Stuart-Wortley; Eleanor Marx; Wilhelmina Powlett, Duchess of Cleveland; Rachel Russell, Lady Russell; Martha Jefferson; Rowland Hill; Hertha Ayrton; John Manners, 5th Duke of Rutland; Lady Randolph Churchill; Princess Charlotte of Wales; Mary Curzon, Baroness Curzon of Kedleston; Jenny Longuet; Harriet Sutherland-Leveson-Gower, Duchess of Sutherland; Georgiana Spencer, Countess Spencer; Princess Frederica Charlotte of Prussia; Charles Kemble; Emily Temple, Viscountess Palmerston; Caroline Herschel; Princess Marie of Battenberg; Lady Caroline Lamb; Elizabeth Inchbald; Julia Grant; Catherine Howard; Lady Florence Dixie; Mary Berry (writer, born 1763); Nell Arthur; Lady Flora Hastings; Frances Sheridan; Grace Elliott; Catherine of Braganza; Frances Anne Spencer-Churchill, Duchess of Marlborough; Margaret Roper; Princess Augusta of Great Britain; Cornelia Schlosser; Jane Digby

In [24]:
print_neighbors("Alan Turing")

Alan Turing: Bernhard Riemann; Kurt Gödel; John von Neumann; Herbert A. Simon; John Forbes Nash Jr.; Claude Shannon; Nikolai Lobachevsky; Benoit Mandelbrot; Willard Van Orman Quine; Niels Henrik Abel; Frank P. Ramsey; Hermann Minkowski; John Horton Conway; L. E. J. Brouwer; Gottlob Frege; Jan Łukasiewicz; Alfred Tarski; Jean Dieudonné; Hilary Putnam; Edmund Landau; Srinivasa Ramanujan; Emmy Noether; George Dantzig; B. F. Skinner; Évariste Galois; Alonzo Church; Øystein Ore; Dennis Ritchie; C. I. Lewis; André Weil; Georg Cantor; Harold Davenport; John Maynard Smith; John Edensor Littlewood; Carl Gustav Hempel; Giuseppe Peano; John Tukey; Rosalind Franklin; Kazimierz Twardowski; Lars Ahlfors; Erwin Schrödinger; Andrey Nikolayevich Tikhonov; Julian Schwinger; Saul Kripke; Eugen Dühring; Aleksandr Lyapunov; John Pople; Paul Cohen; Karl Pearson; Stefan Banach


In [25]:
print_neighbors("Fields Medal")

Fields Medal: IEEE Medal of Honor; Kavli Prize; Rosenstiel Award; Paul Ehrlich and Ludwig Darmstaedter Prize; Albert Einstein World Award of Science; Canada Gairdner International Award; Earle K. Plyler Prize for Molecular Spectroscopy; Dannie Heineman Prize for Mathematical Physics; Bôcher Memorial Prize; NAS Award in Chemical Sciences; Henri Poincaré Prize; Sir George Stokes Award; Margaret Edwards Award; Naguib Mahfouz Medal for Literature; Lord Lewis Prize; Royal Gold Medal; Crafoord Prize; Bruce Medal; Enrico Fermi Award; Albert Einstein Medal; Davisson–Germer Prize in Atomic or Surface Physics; Dannie Heineman Prize for Astrophysics; Henry Draper Medal; IEEE Edison Medal; Fernand Holweck Medal and Prize; Franklin Institute Awards; Albert Lasker Award for Basic Medical Research; Stanley J. Korsmeyer Award; Perkin Prize for Organic Chemistry; Lasker-DeBakey Clinical Medical Research Award; Nansen Refugee Award; Elie Cartan Prize; Max Planck Medal; Lilienfeld Prize; Cole Prize; Oliv

In [26]:
print_neighbors("Gracilinanus")

Gracilinanus: Scolomys; Aethalops; Oligoryzomys; Raphicerus; Vesper mouse; Rhabdomys; Balantiopteryx; Arielulus; Bassariscus; Microsciurus; Balearic shrew; Glauconycteris; Lophostoma; Rhogeessa; Platyrrhinus; Neacomys; Molossops; Eomeropidae; Opluridae; Thalpomys; Thyroptera; Cynomops; Paroedura; Procapra; Gerbillus; Eupleres; Bog lemming; Chaetophractus; Tremarctos; Dorcopsis (genus); Cabassous; Blarinella; Sturnira; Alticola; Euneomys; Talapoin; Geoxus; Dwarf lemur; Neusticomys; Microryzomys; Nesokia; Baiomys; Eothenomys; Tetraphalerus; Petromyscus; Proedromys; Dyacopterus; Mormoops; Akodon; Pecari


In [27]:
print_neighbors("Efren Reyes")

Efren Reyes: Baguio Wong; Kuo Po-cheng; Chao Fong-pang; Rex Johnston (bowls); Wu Jia-qing; Tom Storm; Nick Varner; Lin Yueh-ping; Cao Yanhua; Yenebier Guillén; Bai Yawen; Pirata Morgan Jr.; Urta Rozenstruik; Buddy Hall; Chen Siyi; Jessica Matassa; Jenny Johnson (field hockey); Kamil Aliyev; Kerrie Engel; Jose Parica; Antonio Lining; Rodolfo Luat; Tharatip Sridee; Darcy Thompson (cyclist); Zhang Jie (fencer); Janaina Cunha; Sun Qiang; Krystal Rota; Ge Xin'ai; Donna Ritchie; Swati Singh; Amberlynn Weber; Ray Davies (sailor); Alireza Katiraei; Dominic Collins (swimmer); Ali Forsyth; Taeja James; Skyler Woodward; Lu Wei (diver); Anatoliy Moroz; Mark Rohan; Max Eberle; Li Hewen; Wang Qun (swimmer); TJ Schiller; Mike Massey; Xie Saike; Salom Yiu; Sonika Kaliraman; Ronato Alcano


In [28]:
print_neighbors("Veliko Tarnovo")

Veliko Tarnovo: Kluczbork; Yambol; Kyustendil; Targovishte; Pančevo; Kragujevac; Ruse, Bulgaria; Sombor; Vratsa; Pazardzhik; Vidin; Gorna Oryahovitsa; Herceg Novi; Lovech; Dupnitsa; Tryavna; Dobrich; Smederevo; Valjevo; Ukmergė; Vršac; Haskovo; Karakol; Svishtov; Petrich; Inowrocław; Sabinov; Gabrovo; Belogradchik; Pernik; Novi Pazar; Kardzhali; Asenovgrad; Spišské Podhradie; Dimitrovgrad, Bulgaria; Kazanlak; Pezinok; Heide; Karlovo; Srinagar; La Serena, Chile; Veliki Preslav; Partizánske; Montana, Bulgaria; Solvychegodsk; Vranov nad Topľou; Prijepolje; Shumen; Kraljevo; Viña del Mar
